In [2]:
!pip install torchinfo --quiet
!pip install torchtext --quiet
!pip install pdfplumber --quiet

In [3]:
# Importing libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import random
import time
import os
from tqdm.notebook import tqdm
from collections import Counter
import string
from typing import *

In [58]:
!wget https://kvongcmehsanalibrary.wordpress.com/wp-content/uploads/2021/07/harry-potter-sorcerers-stone.pdf
!wget https://kvongcmehsanalibrary.wordpress.com/wp-content/uploads/2021/07/harry-potter-and-the-goblet-of-fire.pdf
!wget https://kvongcmehsanalibrary.wordpress.com/wp-content/uploads/2021/07/harrypotter.pdf
!wget https://kvongcmehsanalibrary.wordpress.com/wp-content/uploads/2021/07/harry-potter-and-the-half-blood-prince-j.k.-rowling.pdf

--2024-11-25 17:44:54--  https://kvongcmehsanalibrary.wordpress.com/wp-content/uploads/2021/07/harry-potter-sorcerers-stone.pdf
Resolving kvongcmehsanalibrary.wordpress.com (kvongcmehsanalibrary.wordpress.com)... 192.0.78.12, 192.0.78.13
Connecting to kvongcmehsanalibrary.wordpress.com (kvongcmehsanalibrary.wordpress.com)|192.0.78.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1049997 (1.0M) [application/pdf]
Saving to: ‘harry-potter-sorcerers-stone.pdf’

harry-potter-sorcer 100%[===================>]   1.00M  5.49MB/s    in 0.2s    

2024-11-25 17:44:54 (5.49 MB/s) - ‘harry-potter-sorcerers-stone.pdf’ saved [1049997/1049997]

--2024-11-25 17:44:54--  https://kvongcmehsanalibrary.wordpress.com/wp-content/uploads/2021/07/harry-potter-and-the-goblet-of-fire.pdf
Resolving kvongcmehsanalibrary.wordpress.com (kvongcmehsanalibrary.wordpress.com)... 192.0.78.12, 192.0.78.13
Connecting to kvongcmehsanalibrary.wordpress.com (kvongcmehsanalibrary.wordpress.com)|192.

In [4]:
# # Create Custom datasets

# class TextDataset(Dataset):
#   def __init__(
#       self,
#       text: List[str],
#       tokenizer_name: str = 'gpt2',
#       max_length: int = 128,
#       stride: int = 1
#       ):

#     self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
#     self.max_length = max_length
#     self.stride = stride
#     self.texts = text

#     # Add special tokens is they don't exist

#     special_tokens = {
#         'pad_token': "<pad>",
#         'bos_token': "<bos>",
#         "eos_token": "<eos>",
#         "mask_token": "<mask>",
#         "unk_token": "<unk>"
#     }

#     # Add special tokens that don't exist in the Tokenizer

#     special_tokens_dict = {}

#     for token_type, token in special_tokens.items():
#       if not hasattr(self.tokenizer, token) or getattr(self.tokenizer, token) is None:
#         special_tokens_dict[token_type] = token


#     if special_tokens_dict:
#       self.tokenizer.add_special_tokens(special_tokens_dict)

#     self.chunk_mapping = self._create_chunk_mapping(text)


#   def _create_chunk_mapping(self, texts: List[str]) -> List[Tuple[int, int]]:
#     """create a mapping of global index to (text_idx, chunk_idx)"""
#     chunk_mapping = []

#     for text_id, text in tqdm(enumerate(texts)):

#       tokenized = self.tokenizer.encode(text, add_special_tokens=True, max_length=1024, truncation=True)

#       if len(tokenized) > 0:
#         num_chunks = max(0, (len(tokenized) - self.max_length) // self.stride + 1)
#       else:
#         continue

#       for chunk_idx in range(num_chunks):
#         chunk_mapping.append((chunk_idx, text_id))

#     print(f"Created {len(chunk_mapping)} chunks from {len(texts)} texts.")
#     return chunk_mapping


#   def __len__(self) -> int:
#     return len(self.chunk_mapping)

#   def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
#     if idx >= len(self.chunk_mapping):
#       raise IndexError("idx out of bound")

#     chunk_idx, text_id = self.chunk_mapping[idx]

#     text = self.texts[text_id]

#     chunk = self.tokenizer.encode(text, add_special_tokens=True, max_length=self.max_length + 1, truncation=True)

#     if len(chunk) < self.max_length:
#       chunk += [self.tokenizer.pad_token_id] * (self.max_length - len(chunk))

#     input_ids = chunk[:-1]
#     label = chunk[1:]

#     attention_mask = [1] * (len(input_ids)) + [0] * (self.max_length - len(input_ids))

#     return {
#         'input_ids': torch.tensor(input_ids, dtype=torch.long),
#         'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
#         'labels': torch.tensor(label, dtype=torch.long)
#       }


# def create_dataloaders(
#     texts: List[str],
#     batch_size: int = 256,
#     train_split: float = 0.9,
#     seed: int = 42) -> Tuple[DataLoader, DataLoader]:

#     if not texts:
#       raise ValueError("text cannot be empty")

#     """
#     create train and validation dataloader from given ratio
#     """

#     print(f"Total number of document is passed: {len(texts)}")

#     # Filterout empty text
#     texts = [text for text in texts if isinstance(text, str) and text.strip()]

#     print(f"Total number of texts after filtering: {len(texts)}")

#     random.seed(seed)

#     random.shuffle(texts)

#     train_size = int(train_split * len(texts))

#     train_texts = texts[:train_size]
#     val_texts = texts[train_size:]

#     # Create Dataset
#     train_dataset = TextDataset(train_texts)
#     val_dataset = TextDataset(val_texts)


#     # Create train loader
#     train_loader = DataLoader(
#         train_dataset,
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=2,
#         collate_fn=lambda x: x
#     )

#     # Create test loader
#     val_loader = DataLoader(
#         val_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=2,
#         collate_fn=lambda x: x
#     )

#     return train_loader, val_loader

In [5]:
# I have taken a single book of harry-potter
import pdfplumber

book_list = ['/content/harry-potter-sorcerers-stone.pdf', '/content/harry-potter-and-the-goblet-of-fire.pdf', '/content/harry-potter-and-the-half-blood-prince-j.k.-rowling.pdf', '/content/harrypotter.pdf']
pages = []
for book in tqdm(book_list):
    with pdfplumber.open(book) as pdf:
      try:
        for page in tqdm(pdf.pages[:1000]):
          text = page.extract_text()
          pages.append(text)
      except Exception as e:
        print(f"Error {e}")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/221 [00:00<?, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]

  0%|          | 0/552 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
# Create Custom datasets

class Tokenizer:
  def __init__(self):
    self.word2index = {'<pad>': 0, '<sos>': 1,'eos': 2,'<unk>': 3}
    self.index2word = {0: '<pad>', 1: '<sos>', 2: 'eos', 3: '<unk>'}
    self.word_nums = 4

  def encode(self, text: str, max_length=None) -> List:
    encoding = []
    for word in text.split():
      if word not in self.word2index:
        self.word2index[word] = self.word_nums
        self.index2word[self.word_nums] = word
        encoding.append(self.word_nums)
        self.word_nums += 1
      else:
        encoding.append(self.word2index[word])
    if max_length:
      return encoding[:max_length]
    return encoding

  def decode(self, input_ids):
    if isinstance(input_ids, torch.Tensor):
      input_ids = input_ids.numpy()
    text = [self.index2word.get(ids, self.index2word[self.word2index['<unk>']]) for ids in input_ids]
    return " ".join(text)


# Create Dataset for Model
class TextDataset(Dataset):
  def __init__(
      self,
      text: List[str],
      max_length: int = 128,
      stride: int = 16
      ):

    self.max_length = max_length
    self.stride = stride
    self.texts = text
    self.tokenizer = Tokenizer()

    self.chunk_mapping = self._create_chunk_mapping(text)


  def _create_chunk_mapping(self, texts: List[str]) -> List[Tuple[int, int]]:
    """create a mapping of global index to (text_idx, chunk_idx)"""
    chunk_mapping = []

    for text_id, text in tqdm(enumerate(texts)):
      tokenized = self.tokenizer.encode(text, max_length=550)

      if len(tokenized) > 0:
        num_chunks = max(0, (len(tokenized) - self.max_length) // self.stride + 1)
      else:
        continue

      for chunk_idx in range(num_chunks):
        chunk_mapping.append((chunk_idx, text_id))

    print(f"Created {len(chunk_mapping)} chunks from {len(texts)} texts.")
    return chunk_mapping

  def __len__(self) -> int:
    return len(self.chunk_mapping)

  def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
    if idx >= len(self.chunk_mapping):
      raise IndexError("idx out of bound")

    chunk_idx, text_id = self.chunk_mapping[idx]

    text = self.texts[text_id]

    chunk = self.tokenizer.encode(text, max_length=self.max_length + 1)

    if len(chunk) < self.max_length:
      chunk += [self.tokenizer.pad_token_id] * (self.max_length - len(chunk))

    input_ids = chunk[:-1]
    label = chunk[1:]

    attention_mask = [1] * (len(input_ids)) + [0] * (self.max_length - len(input_ids))

    return {
        'input_ids': torch.tensor(input_ids, dtype=torch.long),
        'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
        'labels': torch.tensor(label, dtype=torch.long)
      }



# Create DataLoader
def create_dataloaders(
    texts: List[str],
    batch_size: int = 256,
    train_split: float = 0.9,
    seed: int = 42) -> Tuple[DataLoader, DataLoader]:

    if not texts:
      raise ValueError("text cannot be empty")

    """
    create train and validation dataloader from given ratio
    """

    print(f"Total number of document is passed: {len(texts)}")

    # Filterout empty text
    texts = [text for text in texts if isinstance(text, str) and text.strip()]

    print(f"Total number of texts after filtering: {len(texts)}")

    random.seed(seed)

    random.shuffle(texts)

    train_size = int(train_split * len(texts))

    train_texts = texts[:train_size]
    val_texts = texts[train_size:]

    # Create Dataset
    train_dataset = TextDataset(train_texts)
    val_dataset = TextDataset(val_texts)

    # Create train loader
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        collate_fn=lambda x: x
    )

    # Create test loader
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        collate_fn=lambda x: x
    )

    return train_loader, val_loader

In [7]:
# download the 20-newsgroups dataset from kaggle
# !kaggle datasets download -d crawford/20-newsgroups

In [8]:
# # unzip the downloader file
# import zipfile
# zip_ref = zipfile.ZipFile('/content/20-newsgroups.zip', 'r')
# zip_ref.extractall('20-newsgroups')

In [9]:
# # list of all the news group
# news_groups = os.listdir('20-newsgroups')

In [10]:
# def creat_documents(news_groups, path):
#   """
#   function for split every news group corpus into multiple chunks
#   """
#   documents = []
#   for news_group in news_groups:
#     with open(os.path.join(path, news_group), 'r', encoding='latin-1') as f:
#       text = f.read()
#       print('Newsgroup: ' + news_group[:-4])
#     document_list = text.split('Newsgroup: ' + news_group[:-4])
#     document_list = [document for document in document_list if len(document)>10]
#     documents.extend(document_list)
#   return documents

# documents = creat_documents(news_groups, '/content/20-newsgroups')

In [11]:
# print(f'total number of documents is {len(documents)}')

In [7]:
# function for clean the data
def clean_document(document):
    import re
    # Lowercase the document
    document = document.lower()
    # Remove email IDs
    document = re.sub(r'\S+@\S+', '', document)

    # Remove lines starting with specific headers
    document = re.sub(r'^(from|subject|summary|keywords|date|document_id|in article):\s*.*$', '', document, flags=re.MULTILINE)
    # Remove all punctuations
    document = re.sub(r'[^\w\s]', '', document)

    # Remove footers (lines with repeated underscores or patterns indicating footer)
    document = re.sub(r'_{2,}.*', '', document, flags=re.MULTILINE)

    # Remove multiple spaces and replace with single space
    document = re.sub(r'\s+', ' ', document)

    # Remove numbers
    document = re.sub(r"\d+", "", document)

    # Remove leading and trailing whitespace
    document = document.strip()

    return document

In [8]:
# apply function to clean the documents
documents = [clean_document(document) for document in pages if len(document) > 20]

In [9]:
print(f'total number of documents is {len(documents)}')

total number of documents is 1840


In [10]:
# create data loaders
train_loader, val_loader = create_dataloaders(documents)

Total number of document is passed: 1840
Total number of texts after filtering: 1840


0it [00:00, ?it/s]

Created 19595 chunks from 1656 texts.


0it [00:00, ?it/s]

Created 2180 chunks from 184 texts.


In [11]:
# creating a global tokenizer
tokenizer = Tokenizer()
for doc in documents:
  tokenizer.encode(doc)

In [12]:
print(train_loader.dataset[1101]['input_ids'])
print("--------------------------------------------------------------")
print(train_loader.dataset[1101]['labels'])

tensor([ 251,   58,  159,  344, 4472,   34, 1028, 1614,   23,   24,  294,   92,
         962,   70,   53, 1115, 4473,  971,  972,  103, 1693,  144,  106,   65,
           9, 4474, 1021,   52,   53, 1614,  973,  974, 1190,   27, 3002,    8,
         103,  165,  954, 1275,  106,   65,   69, 1129, 4475,   24,  294,   94,
        3869,    8, 1560,    6, 1231, 4476,  476,   58, 2611,   24,  152,  299,
          52,   53, 3517,  168,  664,   97, 4477,  154,  836,   66,   58,  263,
         312,   65,   71, 2765, 1631,   70,    9,  615,    9,  966,  909,  166,
           6,  639,   70,   34, 1293,  106, 1083,   58,  159,  723,  154,   71,
         585, 4478, 4479,   70,    9,  354,   24,  639,   58,   97,   71,  567,
           9, 4479,   65, 2842,   52,   75,  294,  293, 4480, 4481, 4482,   70,
         791,  676,  737,   80,   24,  246,  973,  974])
--------------------------------------------------------------
tensor([  58,  159,  344, 4472,   34, 1028, 1614,   23,   24,  294,   92,  962,


In [13]:
print(tokenizer.decode(train_loader.dataset[1101]['input_ids']))
print("--------------------------------------------------------------")
print(tokenizer.decode(train_loader.dataset[1101]['labels']))

thinking mr away always scurried perfectly astride slid of number dressed so bring beefy didnt shoulders mouthed pointing less small crutches shuddered their which sorcerers petrified louder just didnt slid kicking mother n drive malkins the small woke hands question their which big takin gallery number dressed time chops the coils potter demanded suits passersby mr didn number seen saw just didnt sprouts gray oh garden tripped this streets made mr looking huddle which man grubbylooking swearing beefy sorcerers britain sorcerers mean is up potter thoughtmaybeit beefy perfectly class their wed mr away belt this man weatherman suit clanging beefy sorcerers daylight number thoughtmaybeit mr garden man sunrise sorcerers clanging which shrink just although dressed strangely crashing sprinted doorpost beefy exactly front crooked thin number tabby kicking mother
--------------------------------------------------------------
mr away always scurried perfectly astride slid of number dressed so b

In [14]:
# Model Building

class LSTMTextGeneration(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout=0.3):
    super(LSTMTextGeneration, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
    self.fc = nn.Linear(hidden_dim, vocab_size)

  def forward(self, x):
    # Input: (batch_size, seq_len)

    x = self.embedding(x) ## (batch_size, vocab_size, embedding_dim)
    x, hidden = self.lstm(x) ## (batch_size, seq_len, hidden_dim)
    x = self.fc(x) ## (batch_size, seq_len, vocab_size)
    return x, hidden

In [15]:
# load the model
vocab_size = len(tokenizer.word2index)
print(vocab_size)
embedding_dim = 128
hidden_dim = 256
num_layers = 2
dropout = 0.2

with torch.no_grad():
  model = LSTMTextGeneration(vocab_size, embedding_dim, hidden_dim, num_layers, dropout)

17269


In [16]:
# Model summary
summary(model)

Layer (type:depth-idx)                   Param #
LSTMTextGeneration                       --
├─Embedding: 1-1                         2,210,432
├─LSTM: 1-2                              921,600
├─Linear: 1-3                            4,438,133
Total params: 7,570,165
Trainable params: 7,570,165
Non-trainable params: 0

In [17]:
# testing the model before training
test = train_loader.dataset[0]['input_ids'].unsqueeze(0)
logits, _ = model(test)
tokenizer.decode(torch.argmax(logits.squeeze(0), dim=-1))

'dishonest contentedly malfoys medal medal medal medal medal medal medal hepzibah aproned aproned buildings essay medal medal medal medal medal medal newsprint aproned aproned enough medal medal medal are inflexible aproned aproned aproned figuration figuration figuration figuration wasnt chortling hepzibah inflexible into are are hepzibah disturbance hepzibah medal lifes severus frogs mcla medal medal medal medal figuration medal medal figuration figuration medal whove whove medal medal medal medal medal figuration essay darkened darkened aproned exercise maddened maddened bumping whove carts carts decrepitlooking whimpering aproned aproned toadstools aproned wail wail wasnt aproned figuration figuration medal wasnt medal medal dyou exercise medal discussing retaliating scored yellowish maddened medal inflexible enough aproned squa aproned medal aproned figuration figuration figuration firenze fu fu severus severus dyou darkened shortest darkened shortest aproned medal'

In [18]:
logits.view(-1, logits.size(-1)).shape

torch.Size([128, 17269])

In [ ]:
# Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 10

history = {
    "train_loss": [],
    "val_loss": []
}

# Training
for epoch in range(epochs):

  # Traning phase
  model.train()
  total_train_loss = 0
  total_val_loss = 0.

  train_bar = tqdm(train_loader, desc = f'Epoch {epoch + 1} Traning')
  for idx, batch in enumerate(train_bar):

    # load input ids and labels
    input_ids = [x['input_ids'] for x in batch]
    labels = [x['labels'] for x in batch]

    # bring data at cuda
    input_ids = torch.stack(input_ids).to(device)
    labels = torch.stack(labels).to(device)

    # setup all privios gradient to zero
    optimizer.zero_grad()
    # forward pass
    logits, _ = model(input_ids)

    # claculating loss
    loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))

    # backward pass
    loss.backward()
    optimizer.step()

    if idx % 100 == 0:
      train_bar.set_postfix({"Train Loss": loss.item()})

    total_train_loss += loss.item()

  # Validation

  with torch.no_grad():

    val_bar = tqdm(val_loader, desc = f'Epoch {epoch + 1} validation')
    for idx, batch in enumerate(val_bar):

      # load input ids and labels
      input_ids = [x['input_ids'] for x in batch]
      labels = [x['labels'] for x in batch]

      # bring data at cuda
      input_ids = torch.stack(input_ids).to(device)
      labels = torch.stack(labels).to(device)

      # setup all privios gradient to zero
      optimizer.zero_grad()
      # forward pass
      logits, _ = model(input_ids)

      # claculating loss
      loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))

      if idx % 50 == 0:
        val_bar.set_postfix({"val Loss": loss.item()})

      total_val_loss += loss.item()

  history['train_loss'].append(total_train_loss)
  history['val_loss'].append(total_val_loss)

Epoch 1 Traning:   0%|          | 0/77 [00:00<?, ?it/s]

Epoch 1 validation:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 2 Traning:   0%|          | 0/77 [00:00<?, ?it/s]

Epoch 2 validation:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 3 Traning:   0%|          | 0/77 [00:00<?, ?it/s]

Epoch 3 validation:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 4 Traning:   0%|          | 0/77 [00:00<?, ?it/s]

In [55]:
# testing the model before training
test = val_loader.dataset[500]['input_ids'].unsqueeze(0).to(device)
logits, _ = model(test)
tokenizer.decode(torch.argmax(logits.squeeze(0), dim=-1).cpu().numpy())

'perfectly somebody dumbledore rude be called hushed somebody or perfectly soon director or privet are are are are neck weirdos last last perfectly drawn last or or be overhead or or thank starts or or called or stone stone stone called thanks thank theyre yourself thank why neck weirdos last neck normal thin drills rushed soon director nerve things no no no no strange stone soon people together wont sorcerers jk stone thank stone thank stone stone stone neck say starts well good good sorcerers well well privet privet called stone stone stone jk eight silly eight good good sorcerers stone stone good be stone stone normal stone stone stone stone thank or stone outside weirdos being last had stone no thank privet light no fourposters lily blinked'

In [42]:
tokenizer.decode(val_loader.dataset[500]['input_ids'])

'came pecked cheek tried kiss goodbye who missed now picked having tantrum man four throwing cereal walls little tyke mrs chortled although left who house that got you jk just car backed fours corner first sign house twice sign came this who beefy something out peculiar cat reading that house very anyone past map who second mrs realize jk then hardly jerked came head mrs having around look again tabby tabby again dursleys standing mrs wasnt son sight been corner thinking must trick you you very light blinked just again stared house that starts m although back finer drove you this road watched son perfectly mirror just tried kiss said looking cats blinked finer got blinked just again couldnt hardly read maps signs perfectly who gave himself jk'